In [ ]:
import os
import sys
sys.path.append("../")

import pandas as pd
import numpy as np
import datetime as dt
import re

import altair as alt


In [ ]:
BOXID = [
    "133.195-1",
    "027.4090-1",
    "133.134-1",
    "178.518-1",
    "053.746-1",
    "133.141-1",
    "027.0380-1",
    "133.301-1",
    "027.2100-1",
    "027.4390-1",
]

In [ ]:
i=2
df_sample = (
    pd.read_parquet(path = f"../data/raw/BOXID={BOXID[i]}/L=sumli/")
    .query("TOP == 1")
    .sort_values(["YEAR", "WEEK"])
)
df_sample["DATE"] =  df_sample.apply(lambda d: dt.datetime.fromisocalendar(d["YEAR"], d["WEEK"], 1), axis=1)
df_sample

In [ ]:
(
    alt.Chart(df_sample)
    .mark_line()
    .encode(
        alt.X("DATE:T", title="date"),
        alt.Y("VALUE:Q", title=f"P sumli"),
    )
    .properties(width=900)
)